In [2]:
import pandas as pd
import numpy as np
import pyarrow 

In [3]:
import seaborn as sns

### 1. Extracción de datos
Leer el archivo reservas_hoteles.parquet para obtener la información relevante de hoteles de la competencia.

In [4]:
df_eraw = pd.read_parquet('../data/reservas_hoteles.parquet')
df_eraw

,id_reserva,id_cliente,nombre,apellido,mail,competencia,fecha_reserva,inicio_estancia,final_estancia,id_hotel,precio_noche,nombre_hotel,estrellas,ciudad
0,5256cc90-139b-43d2-8ec5-412495d751cf,346f2a77-33f4-4b2d-82f3-e8501e21b28d,Feliciana,Cantón,feliciana.cantón@example.com,True,,2025-03-01,2025-03-02,113,NaN,,NaN,
1,84fd6209-bd8d-4a92-bfe6-c68ee2c49271,346f2a77-33f4-4b2d-82f3-e8501e21b28d,Leonardo,Hierro,leonardo.hierro@example.com,True,,2025-03-01,2025-03-02,194,NaN,,NaN,
2,40c4cb55-d1f5-407b-832f-4756b8ff77b4,6ec4d7d2-e382-47cc-94f6-c647577d4035,Maite,Calatayud,maite.calatayud@example.com,False,2025-02-09,2025-03-01,2025-03-02,14,119.96,Hotel Monte Verde,1.0,Madrid
3,f2ce8df5-7844-43e1-8c0f-97ce7a208f21,aed17fe8-eb18-4f69-bf08-df1afdc339c3,Tecla,Bonet,tecla.bonet@example.com,False,2025-02-08,2025-03-01,2025-03-02,38,211.53,Hotel Brisas del Mar,5.0,Madrid
4,57d4515a-447d-4067-afcb-9bdcf4e4e915,710a2fbb-264f-426a-aa66-f8fffd146a2d,Amílcar,Andrés,amílcar.andrés@example.com,False,2025-02-02,2025-03-01,2025-03-02,35,378.37,Hotel Camino del Sol,1.0,Madrid
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15093,49c9a352-a260-46fb-9edc-a2c2d05fdfb9,cc50c0c1-a589-44b7-8a41-2c731d033f89,Victoriano,Miralles,victoriano.miralles@example.com,False,2025-02-02,2025-03-01,2025-03-02,36,367.65,Hotel Las Estrellas,3.0,Madrid
15094,f07370e3-feeb-4893-a9ff-4ec4fb518a55,2bf0215f-7f7b-4cfb-9a37-d2022c492410,Anacleto,Toledo,anacleto.toledo@example.com,False,2025-02-07,2025-03-01,2025-03-02,38,144.35,Gran Hotel Madrid,4.0,Madrid
15095,1d4b9293-1733-45ce-8a8e-79d3ac4694eb,4ba34638-c43e-455f-886d-b3e3dd223a07,Máxima,Mayoral,máxima.mayoral@example.com,False,2025-02-08,2025-03-01,2025-03-02,29,463.32,Hotel Jardines del Rey,2.0,Madrid
15096,d6823ca7-e236-48b1-bf68-6b88d9b6534a,a017cb9c-160b-4b90-b4a9-4bdb2dce546a,Rafa,Alcaraz,rafa.alcaraz@example.com,False,2025-02-10,2025-03-01,2025-03-02,15,247.63,Hotel Palacio Imperial,1.0,Madrid


In [5]:
df = df_eraw.copy()

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15098 entries, 0 to 15097
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id_reserva       15098 non-null  object 
 1   id_cliente       15098 non-null  object 
 2   nombre           15098 non-null  object 
 3   apellido         15098 non-null  object 
 4   mail             15098 non-null  object 
 5   competencia      15098 non-null  bool   
 6   fecha_reserva    15098 non-null  object 
 7   inicio_estancia  15023 non-null  object 
 8   final_estancia   15023 non-null  object 
 9   id_hotel         15098 non-null  int64  
 10  precio_noche     9874 non-null   float64
 11  nombre_hotel     15098 non-null  object 
 12  estrellas        9926 non-null   float64
 13  ciudad           15098 non-null  object 
dtypes: bool(1), float64(2), int64(1), object(10)
memory usage: 1.5+ MB


Pasamos a formato fecha todas las columnas de fechas que están en otros formatos.

In [7]:
def tran_data(lista_col, dataframe):
    for col in lista_col:
        dataframe[col] = pd.to_datetime(dataframe[col])
    return dataframe

In [8]:
col_fechas = ["fecha_reserva", "inicio_estancia", "final_estancia"]
df = tran_data(col_fechas, df)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15098 entries, 0 to 15097
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   id_reserva       15098 non-null  object        
 1   id_cliente       15098 non-null  object        
 2   nombre           15098 non-null  object        
 3   apellido         15098 non-null  object        
 4   mail             15098 non-null  object        
 5   competencia      15098 non-null  bool          
 6   fecha_reserva    9926 non-null   datetime64[ns]
 7   inicio_estancia  15023 non-null  datetime64[ns]
 8   final_estancia   15023 non-null  datetime64[ns]
 9   id_hotel         15098 non-null  int64         
 10  precio_noche     9874 non-null   float64       
 11  nombre_hotel     15098 non-null  object        
 12  estrellas        9926 non-null   float64       
 13  ciudad           15098 non-null  object        
dtypes: bool(1), datetime64[ns](3), float64

### Duplicados

Borramos todos las filas que están duplicadas y son exactamente iguales.

In [9]:
df.duplicated().sum()

np.int64(98)

In [10]:

df.drop_duplicates(inplace=True)

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15000 entries, 0 to 14999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   id_reserva       15000 non-null  object        
 1   id_cliente       15000 non-null  object        
 2   nombre           15000 non-null  object        
 3   apellido         15000 non-null  object        
 4   mail             15000 non-null  object        
 5   competencia      15000 non-null  bool          
 6   fecha_reserva    9828 non-null   datetime64[ns]
 7   inicio_estancia  14925 non-null  datetime64[ns]
 8   final_estancia   14925 non-null  datetime64[ns]
 9   id_hotel         15000 non-null  int64         
 10  precio_noche     9776 non-null   float64       
 11  nombre_hotel     15000 non-null  object        
 12  estrellas        9828 non-null   float64       
 13  ciudad           15000 non-null  object        
dtypes: bool(1), datetime64[ns](3), float64(2), 

In [12]:
df["id_cliente"].duplicated().sum()

np.int64(153)

### Nulos

Vamos a estudiar cómo son los nulos para ver cómo trabajamos con ellos.

In [13]:
print(df["ciudad"].isnull().sum())  # Contamos los nulos.
print(df["ciudad"].unique())  # Muestra los valores únicos

0
['' 'Madrid']


In [14]:
print(df["ciudad"].isnull().sum())  # Ahora debe mostrar la cantidad correcta de nulos
print(df["ciudad"].unique())  # Ver los valores únicos

0
['' 'Madrid']


In [15]:
df.replace({"ciudad": ""}, np.nan, inplace=True)
df.fillna({"ciudad": "Madrid"}, inplace=True)

In [16]:
df.sample(20)

,id_reserva,id_cliente,nombre,apellido,mail,competencia,fecha_reserva,inicio_estancia,final_estancia,id_hotel,precio_noche,nombre_hotel,estrellas,ciudad
13590,0ad28b4a-b729-41c4-8bda-37801ac21916,fbcccc66-b87b-4e9e-bd34-4addb6c0c33d,Carla,Manuel,carla.manuel@example.com,False,2025-02-05,2025-03-01,2025-03-02,14,334.45,Hotel Mirador Real,1.0,Madrid
472,628770c9-f99d-41f4-a95a-04a78a284f20,009e53f8-6b73-4339-bd26-7e4e3dff1864,Perla,Pablo,perla.pablo@example.com,False,2025-02-01,2025-03-01,2025-03-02,27,61.82,Hotel Camino del Sol,5.0,Madrid
14200,863a9a36-d2a0-4a15-8aac-5fa53188e82c,9143cc23-172f-4d5c-bb7d-69889ce8a9ae,Jose,Izquierdo,jose.izquierdo@example.com,False,2025-02-11,2025-03-01,2025-03-02,49,201.28,Hotel Jardines del Rey,4.0,Madrid
12798,a48b0e46-2b1a-4dc2-aebe-26897e3af80c,cd07a12f-8db8-4e2b-a916-442b161de0bf,Fabio,Adán,fabio.adán@example.com,True,NaT,2025-03-01,2025-03-02,186,NaN,,NaN,Madrid
2180,2eccbdf1-3235-46b6-ac67-e5658a394e8e,82f419de-c538-45d5-a219-44ddc2268e86,Agustín,Aguirre,agustín.aguirre@example.com,False,2025-02-10,2025-03-01,2025-03-02,38,346.20,Hotel Camino del Sol,2.0,Madrid
7860,6de844cc-265a-4452-b8ba-50fa38f94353,81ce15b8-2338-481e-9550-09cdae2b81ac,Roberta,Castillo,roberta.castillo@example.com,True,NaT,2025-03-01,2025-03-02,103,NaN,,NaN,Madrid
9396,ea6df04c-2720-4ef5-bda6-7d4f321218d0,b4145b81-fbc8-4d48-92ac-4f67f6286988,Andrés Felipe,Bou,andrés felipe.bou@example.com,True,NaT,2025-03-01,2025-03-02,194,NaN,,NaN,Madrid
3377,80c37f86-2280-43ba-a776-a9153b1ef3d5,df7bcc5a-c5c4-4695-86dc-09a765cba790,Santos,Milla,santos.milla@example.com,False,2025-02-11,2025-03-01,2025-03-02,36,441.63,Hotel Encanto Real,2.0,Madrid
9977,199e1c90-a13c-44ea-9e06-64ed3cfd6399,b33bceea-c482-424e-8030-42a73e651dda,Joaquina,Galan,joaquina.galan@example.com,False,2025-02-12,2025-03-01,2025-03-02,48,314.46,Hotel Torre Dorada,1.0,Madrid
9754,1fd49822-1022-4b4d-b136-2c5f7b84feb2,e2c28787-0f5b-42c5-8613-5169ba324493,Clara,Boix,clara.boix@example.com,False,2025-02-08,2025-03-01,2025-03-02,43,380.35,Hotel Sol y Luna,3.0,Madrid


In [17]:
df["precio_noche"].describe().T

count    9776.000000
mean      275.066498
std       129.501317
min        50.010000
25%       163.062500
50%       273.420000
75%       387.810000
max       499.820000
Name: precio_noche, dtype: float64

1.	La media (275.12) y la mediana (273.32) son muy cercanas, lo cuál quiere decir que la distribución no está muy sesagada.

2.	La desviación estándar (129.41) es alta, por lo que hay bastante dispersión en los precios.

3.	Los cuartiles muestran una distribución bastante amplia:

	•	25% de los hoteles tienen precios ≤163.12

	•	50% de los hoteles tienen precios ≤273.32
	
	•	75% de los hoteles tienen precios ≤387.83

### Limpieza 

In [18]:
df.columns

Index(['id_reserva', 'id_cliente', 'nombre', 'apellido', 'mail', 'competencia',
       'fecha_reserva', 'inicio_estancia', 'final_estancia', 'id_hotel',
       'precio_noche', 'nombre_hotel', 'estrellas', 'ciudad'],
      dtype='object')

In [24]:
#Vamos a intentar calcular el número de noches que está cada reserva restando la fecha de salida - la fecha de entrada:

df["Tiempo_estancia"] = df["final_estancia"] - df["inicio_estancia"]
df

,id_reserva,id_cliente,nombre,apellido,mail,competencia,fecha_reserva,inicio_estancia,final_estancia,id_hotel,precio_noche,nombre_hotel,estrellas,ciudad,Tiempo_estancia
0,5256cc90-139b-43d2-8ec5-412495d751cf,346f2a77-33f4-4b2d-82f3-e8501e21b28d,Feliciana,Cantón,feliciana.cantón@example.com,True,NaT,2025-03-01,2025-03-02,113,NaN,,NaN,Madrid,1 days
1,84fd6209-bd8d-4a92-bfe6-c68ee2c49271,346f2a77-33f4-4b2d-82f3-e8501e21b28d,Leonardo,Hierro,leonardo.hierro@example.com,True,NaT,2025-03-01,2025-03-02,194,NaN,,NaN,Madrid,1 days
2,40c4cb55-d1f5-407b-832f-4756b8ff77b4,6ec4d7d2-e382-47cc-94f6-c647577d4035,Maite,Calatayud,maite.calatayud@example.com,False,2025-02-09,2025-03-01,2025-03-02,14,119.96,Hotel Monte Verde,1.0,Madrid,1 days
3,f2ce8df5-7844-43e1-8c0f-97ce7a208f21,aed17fe8-eb18-4f69-bf08-df1afdc339c3,Tecla,Bonet,tecla.bonet@example.com,False,2025-02-08,2025-03-01,2025-03-02,38,211.53,Hotel Brisas del Mar,5.0,Madrid,1 days
4,57d4515a-447d-4067-afcb-9bdcf4e4e915,710a2fbb-264f-426a-aa66-f8fffd146a2d,Amílcar,Andrés,amílcar.andrés@example.com,False,2025-02-02,2025-03-01,2025-03-02,35,378.37,Hotel Camino del Sol,1.0,Madrid,1 days
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,58acd6f3-7160-4973-b951-b14188937444,de729428-ec23-4665-a5c9-79e3ec282470,Lorena,Román,lorena.román@example.com,False,2025-02-06,2025-03-01,2025-03-02,35,111.78,Hotel Jardines del Rey,1.0,Madrid,1 days
14996,aa092c3e-7fe6-4985-a5a8-cacd38554b1b,8469b819-d96a-4904-9cdd-710764f6301f,Toño,Narváez,toño.narváez@example.com,True,NaT,2025-03-01,2025-03-02,114,NaN,,NaN,Madrid,1 days
14997,af0f6afa-e7d2-410a-8cdd-5eb62b1e4788,e89751f4-3621-4636-81d3-90e2baa709cb,Nadia,Verdugo,nadia.verdugo@example.com,False,2025-02-11,2025-03-01,2025-03-02,47,442.96,Hotel Brisas del Mar,5.0,Madrid,1 days
14998,6d1f590c-ae87-4582-bc99-9b8cbf1d8e28,c4117358-fd89-44cd-b388-4ec6d3538c72,Custodia,Marco,custodia.marco@example.com,True,NaT,2025-03-01,2025-03-02,194,NaN,,NaN,Madrid,1 days


In [28]:
df["fecha_reserva"].unique()

<DatetimeArray>
[                'NaT', '2025-02-09 00:00:00', '2025-02-08 00:00:00',
 '2025-02-02 00:00:00', '2025-02-10 00:00:00', '2025-02-01 00:00:00',
 '2025-02-12 00:00:00', '2025-02-11 00:00:00', '2025-02-06 00:00:00',
 '2025-02-03 00:00:00', '2025-02-05 00:00:00', '2025-02-07 00:00:00',
 '2025-02-04 00:00:00']
Length: 13, dtype: datetime64[ns]